<a href="https://colab.research.google.com/github/adamstreed/agroecology-basic/blob/main/Genetic_Algorithm_Cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gym
from gym import error, spaces, utils
from gym.utils import seeding
from enum import Enum

class Plant:
    def __init__(self, species, maturity=110):
        self.species = species
        self.maturity = maturity         # consider 'days_to_maturity'
        self.age = 0
        
    def __repr__(self):
        return "{}".format(self.species)
    

class Field(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, size=5, sow_limit=200, season=120, calendar=0):
        # parameters for overall field character
        self.size = size
        self.sow_limit = sow_limit
        self.season = season
        self.calendar = calendar
        
        # constants for computing end-of-season reward---distances represent meters
        self.crowding_dist = .02
        self.maize_maize_dist = .1
        self.bean_support_dist = .1
        self.crowding_penalty = .1
        self.maize_maize_penalty = .9
        self.bean_support_bonus = .6

        self.action_history = []
        self.action_record = 0
        
        # OpenAI action and observation space specifications
        self.action_space = spaces.Discrete(4)
        ### self.observation_space = spaces.???
        
        # field is initialized by calling reset()
        self.field = None
        
    def step(self, action):
        # sow plants (or wait) depending on actions chosen
        # action is an array of n choices; value of n specified in agent code sow_limit
        # could be cleaned up with plants as an enumeration?
        
                   
             ## this part of the code is a work in progress!!   
             ##------------------START OF WIP------------------------------------   
                    
             #declare a new variable that will be the result of the computer figuring out where we want to plant   
             #developed_coord = curr   
             #curr is the result from rollout   
           
             #planttypeTuple = ("Maize", "Bean", "Squash")   
                
             #coordTuple = curr.strip().split()    
                
             #self.field = np.append(self.field, [[self.size*(coordTuple[0]), self.size*(coordTuple[1]), self.size*(coordTuple[2]), Plant(planttypeTuple[0]), Plant(planttypeTuple[1], Plant(planttypeTuple[2]))])   ,
                
            
            ###Experiment: each choice should be represented as an array with 3 elements:
            ### plant choice, y coordinate, x coordinate (in that order).
            ### i.e. action should look like: [[choice1, y1, x1], [choice2, y2, x2]...]
               
        for choice in action:
            if choice[0] == 0:
                self.field = np.append(self.field, [[self.size * choice[1], 
                                                 self.size * choice[2], 
                                                 Plant('Maize')]], axis=0)
            elif choice[0] == 1:
                self.field = np.append(self.field, [[self.size * choice[1], 
                                                 self.size * choice[2], 
                                                 Plant('Bean')]], axis=0)
            elif choice[0] == 2:
                self.field = np.append(self.field, [[self.size * choice[1], 
                                                 self.size * choice[2], 
                                                 Plant('Squash')]], axis=0)
            # when choice == 3, nothing is done (agent waits)   

            #store in history
            if len(self.action_history) < 7:
              self.action_history.append(choice)
            else:
              self.action_history[self.action_record] = choice
            self.action_record = (self.action_record + 1) % 7 
            
        
        # increment timekeeping
        self.calendar +=1
        for plant in self.field:
            plant[2].age += 1

            
        done = self.calendar == self.season
            
        if not done:
            reward = 0
        else:
            reward = self.get_reward()
            
        return self.field, reward, done, self.action_history
    
    def reset(self):
        # field is initialized with one random corn plant in order to make sowing (by np.append) work
        self.field = np.array([[self.size * np.random.random(), 
                                self.size * np.random.random(), 
                                Plant('Maize')]])
        # timekeeping is reset
        self.calendar = 0
        #### MODIFICATION ####
        ## Enables GA Agent to begin interaction ## 
        return self.field
        
    def render(self, mode='human'):
        # initialize plant type arrays so that pyplot won't break if any is empty
        maize = np.array([[None, None]])
        bean = np.array([[None, None]])
        squash = np.array([[None, None]])
        maize_imm = np.array([[None, None]])
        bean_imm = np.array([[None, None]])
        squash_imm = np.array([[None, None]])
        maize_r = 0
        bean_r = 0
        squash_r = 0
        maize_imm_r = 0
        bean_imm_r = 0
        squash_imm_r = 0
        
        # replace initial arrays with coordinates for each plant type; imm are plants that haven't matured
        maize = np.array([row for row in self.field 
                             if row[2].__repr__() == 'Maize' and row[2].age >= row[2].maturity])
        
        for plant in maize:
          maize_r += 1
        bean = np.array([row for row in self.field 
                            if row[2].__repr__() == 'Bean' and row[2].age >= row[2].maturity])
        
        for plant in bean:
          bean_r += 1

        squash = np.array([row for row in self.field 
                              if row[2].__repr__() == 'Squash' and row[2].age >= row[2].maturity])
        
        for plant in squash:
          squash_r += 1
        maize_imm = np.array([row for row in self.field 
                             if row[2].__repr__() == 'Maize' and row[2].age < row[2].maturity])
        
        for plant in maize_imm:
          maize_imm_r += 1

        bean_imm = np.array([row for row in self.field 
                             if row[2].__repr__() == 'Bean' and row[2].age < row[2].maturity])
        
        for plant in bean_imm:
          bean_imm_r += 1

        squash_imm = np.array([row for row in self.field 
                             if row[2].__repr__() == 'Squash' and row[2].age < row[2].maturity])
        
        for plant in squash_imm:
          squash_imm_r += 1
        
        # plot the field---currently breaks if any plant type is absent
        plt.figure(figsize=(10, 10))
        if not(maize_r == 0):
          plt.scatter(maize[:,0], maize[:,1], c='green', s=200, marker = 'o', alpha=.5, edgecolor='#303030')
        if not(bean_r == 0):
          plt.scatter(bean[:,0], bean[:,1], c='brown', s=150, marker = 'o', alpha=.5, edgecolor='#303030')
        if not(squash_r == 0):
          plt.scatter(squash[:,0], squash[:,1], c='orange', s=400, marker = 'o', alpha=.5, edgecolor='#303030')
        if not(maize_imm_r == 0):
          plt.scatter(maize_imm[:,0], maize_imm[:,1], c='green', s=200, marker = 'o', alpha=.1, edgecolor='#303030')
        if not(bean_imm_r == 0):
          plt.scatter(bean_imm[:,0], bean_imm[:,1], c='brown', s=200, marker = 'o', alpha=.1, edgecolor='#303030')
        if not(squash_imm_r == 0):
          plt.scatter(squash_imm[:,0], squash_imm[:,1], c='orange', s=200, marker = 'o', alpha=.1, edgecolor='#303030')

        plt.show()
        
        print("Total yield in Calories is {}.\n---\n".format(round(reward, 1)))
    
    def close(self):
        # unneeded right now? AFAICT this is only used to shut down realtime movie visualizations
        pass
    
    def get_reward(self):
        # array of plant coordinates for computing distances
        xy_array = np.array([[row[0], row[1]] for row in self.field])

        # distances[m,n] is distance from mth to nth plant in field
        distances = np.linalg.norm(xy_array - xy_array[:,None], axis=-1)
        
        reward = 0
        i = 0
        while i < len(self.field):
            if self.field[i,2].age < self.field[i,2].maturity:
                reward += 0
            elif self.field[i,2].__repr__() == 'Maize':
                cal = 1
                j = 0
                while j < len(distances[0]):
                    if (self.field[j,2].__repr__() == 'Bean' 
                            and distances[i,j] < self.bean_support_dist):
                        cal += self.bean_support_bonus
                    if (self.field[j,2].__repr__() == 'Maize' 
                            and i !=j 
                            and distances[i,j] < self.maize_maize_dist):
                        cal *= self.maize_maize_penalty
                    if 0 <= distances[i,j] < self.crowding_dist:
                        cal *= self.crowding_penalty
                    j += 1
                reward += cal
            elif self.field[i,2].__repr__() == 'Bean':
                reward += .25
            elif self.field[i,2].__repr__() == 'Squash':
                reward += 3
            i += 1        
        return reward


# NEAT Code

In [ ]:
!pip install neat-python
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Provided visualization code (by NEAT) ##

import copy
import warnings

import graphviz
import matplotlib.pyplot as plt
import numpy as np


def plot_stats(statistics, ylog=False, view=False, filename='avg_fitness.svg'):
    """ Plots the population's average and best fitness. """
    if plt is None:
        warnings.warn("This display is not available due to a missing optional dependency (matplotlib)")
        return

    generation = range(len(statistics.most_fit_genomes))
    best_fitness = [c.fitness for c in statistics.most_fit_genomes]
    avg_fitness = np.array(statistics.get_fitness_mean())
    stdev_fitness = np.array(statistics.get_fitness_stdev())

    plt.plot(generation, avg_fitness, 'b-', label="average")
    #plt.plot(generation, avg_fitness - stdev_fitness, 'g-.', label="-1 sd")
    plt.plot(generation, avg_fitness + stdev_fitness, 'g-.', label="+1 sd")
    plt.plot(generation, best_fitness, 'r-', label="best")

    plt.title("Population's average and best fitness")
    plt.xlabel("Generations")
    plt.ylabel("Fitness")
    plt.grid()
    plt.legend(loc="best")
    if ylog:
        plt.gca().set_yscale('symlog')

    plt.savefig(filename)
    if view:
        plt.show()

    plt.close()


def plot_spikes(spikes, view=False, filename=None, title=None):
    """ Plots the trains for a single spiking neuron. """
    if plt is None:
        warnings.warn("This display is not available due to a missing optional dependency (matplotlib)")
        return

    t_values = [t for t, I, v, u in spikes]
    v_values = [v for t, I, v, u in spikes]
    u_values = [u for t, I, v, u in spikes]
    I_values = [I for t, I, v, u in spikes]

    fig = plt.figure()
    plt.subplot(3, 1, 1)
    plt.ylabel("Potential (mv)")
    plt.xlabel("Time (in ms)")
    plt.grid()
    plt.plot(t_values, v_values, "g-")

    if title is None:
        plt.title("Izhikevich's spiking neuron model")
    else:
        plt.title("Izhikevich's spiking neuron model ({0!s})".format(title))

    plt.subplot(3, 1, 2)
    plt.ylabel("Recovery (u)")
    plt.xlabel("Time (in ms)")
    plt.grid()
    plt.plot(t_values, u_values, "r-")

    plt.subplot(3, 1, 3)
    plt.ylabel("Current (I)")
    plt.xlabel("Time (in ms)")
    plt.grid()
    plt.plot(t_values, I_values, "r-o")

    if filename is not None:
        plt.savefig(filename)

    if view:
        plt.show()
        plt.close()
        fig = None

    return fig


def plot_species(statistics, view=False, filename='speciation.svg'):
    """ Visualizes speciation throughout evolution. """
    if plt is None:
        warnings.warn("This display is not available due to a missing optional dependency (matplotlib)")
        return

    species_sizes = statistics.get_species_sizes()
    num_generations = len(species_sizes)
    curves = np.array(species_sizes).T

    fig, ax = plt.subplots()
    ax.stackplot(range(num_generations), *curves)

    plt.title("Speciation")
    plt.ylabel("Size per Species")
    plt.xlabel("Generations")

    plt.savefig(filename)

    if view:
        plt.show()

    plt.close()


def draw_net(config, genome, view=False, filename=None, node_names=None, show_disabled=True, prune_unused=False,
             node_colors=None, fmt='svg'):
    """ Receives a genome and draws a neural network with arbitrary topology. """
    # Attributes for network nodes.
    if graphviz is None:
        warnings.warn("This display is not available due to a missing optional dependency (graphviz)")
        return

    if node_names is None:
        node_names = {}

    assert type(node_names) is dict

    if node_colors is None:
        node_colors = {}

    assert type(node_colors) is dict

    node_attrs = {
        'shape': 'circle',
        'fontsize': '9',
        'height': '0.2',
        'width': '0.2'}

    dot = graphviz.Digraph(format=fmt, node_attr=node_attrs)

    inputs = set()
    for k in config.genome_config.input_keys:
        inputs.add(k)
        name = node_names.get(k, str(k))
        input_attrs = {'style': 'filled', 'shape': 'box', 'fillcolor': node_colors.get(k, 'lightgray')}
        dot.node(name, _attributes=input_attrs)

    outputs = set()
    for k in config.genome_config.output_keys:
        outputs.add(k)
        name = node_names.get(k, str(k))
        node_attrs = {'style': 'filled', 'fillcolor': node_colors.get(k, 'lightblue')}

        dot.node(name, _attributes=node_attrs)

    if prune_unused:
        connections = set()
        for cg in genome.connections.values():
            if cg.enabled or show_disabled:
                connections.add(cg.key)

        used_nodes = copy.copy(outputs)
        pending = copy.copy(outputs)
        while pending:
            new_pending = set()
            for a, b in connections:
                if b in pending and a not in used_nodes:
                    new_pending.add(a)
                    used_nodes.add(a)
            pending = new_pending
    else:
        used_nodes = set(genome.nodes.keys())

    for n in used_nodes:
        if n in inputs or n in outputs:
            continue

        attrs = {'style': 'filled', 'fillcolor': node_colors.get(n, 'white')}
        dot.node(str(n), _attributes=attrs)

    for cg in genome.connections.values():
        if cg.enabled or show_disabled:
            #if cg.input not in used_nodes or cg.output not in used_nodes:
            #    continue
            input, output = cg.key
            a = node_names.get(input, str(input))
            b = node_names.get(output, str(output))
            style = 'solid' if cg.enabled else 'dotted'
            color = 'green' if cg.weight > 0 else 'red'
            width = str(0.1 + abs(cg.weight / 5.0))
            dot.edge(a, b, _attributes={'style': style, 'color': color, 'penwidth': width})

    dot.render(filename, view=view)

    return dot

In [ ]:
def field_to_inputs(field, history):
        maize_r = 0
        bean_r = 0
        squash_r = 0
        maize_imm_r = 0
        bean_imm_r = 0
        squash_imm_r = 0
        
        # replace initial arrays with coordinates for each plant type; imm are plants that haven't matured
        maize = np.array([row for row in field 
                             if row[2].__repr__() == 'Maize' and row[2].age >= row[2].maturity])
        
        for plant in maize:
          maize_r += 1

        bean = np.array([row for row in field 
                            if row[2].__repr__() == 'Bean' and row[2].age >= row[2].maturity])
        
        for plant in bean:
          bean_r += 1

        squash = np.array([row for row in field 
                              if row[2].__repr__() == 'Squash' and row[2].age >= row[2].maturity])
        
        for plant in squash:
          squash_r += 1

        maize_imm = np.array([row for row in field 
                             if row[2].__repr__() == 'Maize' and row[2].age < row[2].maturity])
        
        for plant in maize_imm:
          maize_imm_r += 1

        bean_imm = np.array([row for row in field 
                             if row[2].__repr__() == 'Bean' and row[2].age < row[2].maturity])
        
        for plant in bean_imm:
          bean_imm_r += 1

        squash_imm = np.array([row for row in field 
                             if row[2].__repr__() == 'Squash' and row[2].age < row[2].maturity])
        
        for plant in squash_imm:
          squash_imm_r += 1

        data_list = [maize_r, bean_r, squash_r, maize_imm_r, bean_imm_r, squash_imm_r]

        for action in history:
          for num in action:
            data_list.append(num)
        while not(len(data_list) == 27):
          data_list.append(-1)
        return data_list

In [ ]:
import multiprocessing
import os
import pickle

import neat


runs_per_net = 2


# Use the NN network phenotype and the discrete actuator force function.
def eval_genome(genome, config):
    net = neat.nn.FeedForwardNetwork.create(genome, config)

    fitnesses = []

    for runs in range(runs_per_net):
        env = Field()

        # Run the given simulation for up to num_steps time steps.
        fitness = 0.0
        observation = field_to_inputs(env.reset(), [])
        done = False
        while not done:
            action = net.activate(observation)
            for i in range(len(action)):
              action[i] = (action[i] + 1)/2.0
            for i in range(5):
              action[3*i] = ((action[3*i]) * 15) // 4
            action = np.reshape(action, (5,3))
            observation, reward, done, history = env.step(action) 
            observation = field_to_inputs(observation, history)
            fitness += reward

        fitnesses.append(fitness)

    # The genome's fitness is its worst performance across all runs.
    return min(fitnesses)


def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = eval_genome(genome, config)


def run():
    # Load the config file, which is assumed to live in
    # the same directory as this script.
    config_path = '/content/drive/My Drive/agroecology/Config'
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)

    pop = neat.Population(config)
    stats = neat.StatisticsReporter()
    pop.add_reporter(stats)
    pop.add_reporter(neat.StdOutReporter(True))

    pe = neat.ParallelEvaluator(multiprocessing.cpu_count(), eval_genome)
    winner = pop.run(pe.evaluate, 25)

    # Save the winner.
    with open('winner-feedforward', 'wb') as f:
        pickle.dump(winner, f)

    print(winner)

    plot_stats(stats, ylog=True, view=True, filename="feedforward-fitness.svg")
    plot_species(stats, view=True, filename="feedforward-speciation.svg")

    node_names = {-1: 'x', -2: 'dx', -3: 'theta', -4: 'dtheta', 0: 'control'}
    draw_net(config, winner, True, node_names=node_names)

    draw_net(config, winner, view=True, node_names=node_names,
                       filename="winner-feedforward.gv")
    draw_net(config, winner, view=True, node_names=node_names,
                       filename="winner-feedforward-enabled.gv", show_disabled=False)
    draw_net(config, winner, view=True, node_names=node_names,
                       filename="winner-feedforward-enabled-pruned.gv", show_disabled=False, prune_unused=True)


if __name__ == '__main__':
    run()

In [ ]:
with open('winner-feedforward', 'rb') as f:
  c = pickle.load(f)
print("Loaded genome:")
print(c)


config_path = '/content/drive/My Drive/agroecology/Config'
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)

net = neat.nn.FeedForwardNetwork.create(c, config)

env = Field()

fitness = 0.0
observation = field_to_inputs(env.reset(), [])
done = False
while not done:
      action = net.activate(observation)
      for i in range(len(action)):
        action[i] = (action[i] + 1)/2.0
      for i in range(5):
        action[3*i] = (action[3*i] * 15) // 4
      action = np.reshape(action, (5,3))
      print(action)

      observation, reward, done, history = env.step(action) 
      if not done:
        observation = field_to_inputs(observation, history)
      fitness += reward
print("FITNESS:" ,fitness)

print(observation)
print(field_to_inputs(observation, history))
env.render()